In [5]:
cd ../first-order-model

/mnt/disks/user/project/first-order-model


In [6]:
#Libraries Needed
import cv2 
import sys #using for face recognition
import os
from pathlib import Path
import glob
import random
from skimage.util import random_noise ## --> to add noise
import shutil
import numpy as np
import pandas as pd
import zipfile
import keras

#Data Processing
from skimage import io
import os
import glob
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.preprocessing.image import array_to_img
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import random
from PIL import Image
import matplotlib.pyplot as plt
from collections import Counter

#Neural Networks
from tensorflow.python import keras
import tensorflow as tf

#Models
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model

#Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from keras.callbacks import ModelCheckpoint,Callback
from sklearn import metrics
from tensorflow.keras.models import load_model

#Feature Extraction
from demo import load_checkpoints
import torch

/mnt/disks/user/project/first-order-model/demo.py:2: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/mnt/disks/user/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/mnt/disks/user/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/mnt/disks/user/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.

In [7]:
'''
func: unzips the folder
input:
    i.filename: string: value path of the zip file
'''
def unzip_folder(filename):
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('/mnt/disks/user/project/Test/')

In [8]:
#unzip_folder('/mnt/disks/user/project/Test/frames.zip')

### Prepare Dataset

In [9]:
test_set = pd.read_csv('/mnt/disks/user/project/Test/List_of_testing_videos.txt',sep=' ')
test_set.columns = ['label','filepath']

In [10]:
test_set.head()

,label,filepath
0,1,YouTube-real/00208.mp4
1,1,YouTube-real/00063.mp4
2,1,YouTube-real/00024.mp4
3,1,YouTube-real/00021.mp4
4,1,YouTube-real/00036.mp4


In [11]:
test_set.label.value_counts()

0    340
1    177
Name: label, dtype: int64

In [12]:
test_set.filepath = [file.split('/')[-2]+'-'+file.split('/')[-1] for file in test_set.filepath]

In [13]:
test_set.head()

,label,filepath
0,1,YouTube-real-00208.mp4
1,1,YouTube-real-00063.mp4
2,1,YouTube-real-00024.mp4
3,1,YouTube-real-00021.mp4
4,1,YouTube-real-00036.mp4


In [14]:
test_set.label.value_counts()

0    340
1    177
Name: label, dtype: int64

In [15]:
for index, row in test_set.iterrows():
    if row[0] == 0:
        test_set.label[index] = 1
    else:
        test_set.label[index] = 0
    test_set.filepath[index] = test_set.filepath[index].replace('.mp4','.jpg')

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
test_set.label.value_counts()

1    340
0    177
Name: label, dtype: int64

In [17]:
test_set.head()

,label,filepath
0,0,YouTube-real-00208.jpg
1,0,YouTube-real-00063.jpg
2,0,YouTube-real-00024.jpg
3,0,YouTube-real-00021.jpg
4,0,YouTube-real-00036.jpg


### Data Pre-Processing

In [18]:
files = glob.glob('/mnt/disks/user/project/Test/frames/*.jpg', recursive = True)

In [19]:
frame_list = []
for file in files:
    name = file.split('/')[-1]
    frame_list.append(name)

In [20]:
y_test = test_set[test_set['filepath'].isin(frame_list)]
y_test_set = y_test.set_index('filepath')['label'].to_dict()

In [21]:
len(y_test)

516

In [31]:
'''
func: prepare and process batch wise data for training/ablation. 
The image path is converted to image data while the classes are label encoded (0,1,2)
input:
    i.image_list: X data: array of image paths
    ii.classes: y data: dictionary of image paths and their corresponding classes
    iii. feature_extractor model: model used to feature extraction in batches
output:
    i. DataGenerator
'''
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, image_list, classes, feature_extractor, batch_size=32, dim=(256,256), n_channels=3,
                 n_classes=3,shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.classes = classes
        self.image_list = image_list
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.feature_extractor = feature_extractor
        self.shuffle = shuffle
        self.on_epoch_end()
    

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.image_list) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_img_temp = [self.image_list[k] for k in indexes]
        #print(list_img_temp)

        # Generate data
        X, y = self.__data_generation(list_img_temp)
        #print(X,y)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.image_list))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_img_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization of Y
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        #For each image path in the batch: load it, convert it to array, normalize values and pre-process.
        #Append X and respective y values to the empty X and y variables
        for i, ID in enumerate(list_img_temp):
            img = image.load_img(ID, target_size=self.dim)
            img = image.img_to_array(img)
            #img = img/255
            img = preprocess_input(img)
            X[i,] = img
            filename = ID.split('/')[-1]
            y[i] = self.classes[filename]
            
        #Extract features for batch-wise values
        output = torch.tensor(X, dtype=torch.float).to('cuda')
        output_val = encoder(output)
        feature = output_val.cpu().data.numpy().reshape(32,256,256,-1)
        #class encoding to 0,1,2
#         le = LabelEncoder()
#         self.y_value = le.fit_transform(y)
        return feature,keras.utils.to_categorical(y, num_classes=self.n_classes)

In [23]:
config_path='config/vox-256.yaml' #data checkpoints
checkpoint_path='../vox-cpk.pth.tar' #pyTorch Model

In [24]:
#kp_detector model loads keypoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='../vox-cpk.pth.tar')

In [25]:
#get the pre-trained weights from the generator module
encoder = generator.module.bottleneck
encoder.train()

Sequential(
  (r0): ResBlock2d(
    (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (norm1): SynchronizedBatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (norm2): SynchronizedBatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (r1): ResBlock2d(
    (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (norm1): SynchronizedBatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (norm2): SynchronizedBatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (r2): ResBlock2d(
    (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (norm1

### Evaluation - Base Model

In [32]:
base_test_generator = DataGenerator(files, y_test_set,encoder,shuffle=False)

In [33]:
base_eval_base_model = load_model('/mnt/disks/user/project/Test/first_order_binary_best_modelv2.hdf5')

In [34]:
base_predictions = base_eval_base_model.predict_generator(base_test_generator)

In [35]:
base_predictions

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [36]:
base_y_class = np.array(list(base_test_generator.classes.values()))[:len(base_predictions)]
base_y = tf.keras.utils.to_categorical(base_y_class, num_classes=2)

In [37]:
base_prob_max = base_predictions.max(axis=1).reshape(-1, 1)
base_y_pred = np.where(base_predictions == base_prob_max, 1, 0)

In [38]:
base_pred = np.argmax(base_predictions, axis=-1)

In [39]:
len(base_y_pred)

512

In [40]:
len(base_y)

512

In [41]:
Counter(base_pred)

Counter({1: 477, 0: 35})

In [42]:
base_score_test = metrics.accuracy_score(base_y, base_y_pred)
print('Base Model Test Score ',base_score_test)

Base Model Test Score  0.609375


In [43]:
base_cm = confusion_matrix(base_y_class, base_pred)
print(base_cm)

[[ 12 165]
 [ 23 312]]


In [44]:
base_report = classification_report(base_y,base_y_pred)
print(base_report)

             precision    recall  f1-score   support

          0       0.34      0.07      0.11       177
          1       0.65      1.00      0.79       335

avg / total       0.55      0.68      0.56       512



In [45]:
roc_auc_score(base_y, base_y_pred)

0.4997849734378953